![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training Entity Coding Models (SNOMED example)

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.types import StructType, StructField, StringType

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp_jsl.version : 3.1.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.1 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

## Load datasets

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/AskAPatient.fold-0.test.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/AskAPatient.fold-0.train.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/AskAPatient.fold-0.validation.txt
  
dbutils.fs.cp("file:/databricks/driver/AskAPatient.fold-0.test.txt", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/AskAPatient.fold-0.train.txt", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/AskAPatient.fold-0.validation.txt", "dbfs:/")

Out[2]: True

In [0]:
import pandas as pd

cols = ["conceptId","_term","term"]

aap_tr = pd.read_csv("file:/databricks/driver/AskAPatient.fold-0.train.txt",sep="\t",encoding="ISO-8859-1",header=None)
aap_tr.columns = cols
aap_tr["conceptId"] = aap_tr.conceptId.apply(str)

aap_ts = pd.read_csv("file:/databricks/driver/AskAPatient.fold-0.test.txt",sep="\t",header=None)
aap_ts.columns = cols
aap_ts["conceptId"] = aap_ts.conceptId.apply(str)

aap_vl = pd.read_csv("file:/databricks/driver/AskAPatient.fold-0.validation.txt",sep="\t",header=None)
aap_vl.columns = cols
aap_vl["conceptId"] = aap_vl.conceptId.apply(str)

In [0]:
aap_vl.head()

Out[4]:

,conceptId,_term,term
0,267032009,Tired all the time,persisten feeling of tiredness
1,22298006,Myocardial infarction,HEART ATTACK
2,3877011000036101,Lipitor,LIPITOR
3,415690000,Sweating,sweated
4,248491001,Swollen knee,swelling at knee


In [0]:
aap_train_sdf = spark.createDataFrame(aap_tr)
aap_test_sdf = spark.createDataFrame(aap_ts)
aap_val_sdf = spark.createDataFrame(aap_vl)

# Chunk Entity Resolver (Glove Embeddings)

## Create Training Pipeline

In [0]:
document = DocumentAssembler()\
    .setInputCol("term")\
    .setOutputCol("document")

chunk = Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_healthcare_100d", "en", "clinical/models")\
      .setInputCols(["document", "token"])\
      .setOutputCol("embeddings")

chunk_emb = ChunkEmbeddings()\
      .setInputCols("chunk", "embeddings")\
      .setOutputCol("chunk_embeddings")

snomed_training_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    chunk_emb])

snomed_training_model = snomed_training_pipeline.fit(aap_train_sdf)
snomed_data = snomed_training_model.transform(aap_train_sdf).cache()


embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
snomed_extractor = ChunkEntityResolverApproach() \
    .setInputCols("token", "chunk_embeddings") \
    .setOutputCol("recognized") \
    .setNeighbours(1000) \
    .setAlternatives(25) \
    .setNormalizedCol("_term") \
    .setLabelCol("conceptId") \
    .setEnableWmd(True).setEnableTfidf(True).setEnableJaccard(True)\
    .setEnableSorensenDice(True).setEnableJaroWinkler(True).setEnableLevenshtein(True)\
    .setDistanceWeights([1, 2, 2, 1, 1, 1]) \
    .setAllDistancesMetadata(True)\
    .setPoolingStrategy("MAX") \
    .setThreshold(1e32)

In [0]:
model = snomed_extractor.fit(snomed_data)

## Prediction Pipeline

In [0]:
prediction_Model = PipelineModel(stages=[snomed_training_model, model])

In [0]:
aap_train_pred= prediction_Model.transform(aap_train_sdf).cache()
aap_test_pred= prediction_Model.transform(aap_test_sdf).cache()
aap_val_pred= prediction_Model.transform(aap_val_sdf).cache()

In [0]:
aap_test_pred.selectExpr("conceptId","term","_term","recognized[0].result","recognized[0].metadata.resolved_text","recognized[0].metadata.all_k_resolutions").show(truncate=50)

+----------------+--------------------------------+------------------------------------+--------------------+-------------------------------------+--------------------------------------------------+
 conceptId| term| _term|recognized[0].result|recognized[0].metadata[resolved_text]| recognized[0].metadata[all_k_resolutions]|
+----------------+--------------------------------+------------------------------------+--------------------+-------------------------------------+--------------------------------------------------+
 108367008| dislocating| Dislocation of joint| 304297005| Decreased range of shoulder movement|Decreased range of shoulder movement:::Increase...|
3384011000036100| Arthrotec| Arthrotec| 3384011000036100| Arthrotec|Arthrotec:::Celebrex 200 mg capsule: hard:::Cel...|
 166717003| elevated creatine| Serum creatinine raised| 166717003| Serum creatinine raised|Serum creatinine raised:::Increased creatine ki...|
3877011000036101| Lipitor| Lipitor| 3877011000036101| Lipitor|Lipitor:::Lipitor 20 mg tablet: film-coated:::L...|
 402234004| Eczema on feet| Foot eczema| 238539001| Hand eczema|Hand eczema:::Foot eczema:::Cold feet:::Blister...|
 404640003| dizzy| Dizziness| 404640003| Dizziness|Dizziness:::Lightheadedness:::Nausea:::Syncope:...|
 271681002| pain in the pit of my stomach| Stomach ache| 271681002| Stomach ache|Stomach ache:::Abdominal pain:::Myalgia:::Low b...|
 76948002|extreme muscle pain in shoulders| Severe pain| 76948002| Severe pain|Severe pain:::Myalgia/myositis - shoulder:::Mus...|
 36031001| Burning feet| Burning feet| 36031001| Burning feet|Burning feet:::Burning sensation:::Burning pain...|
 76948002| imence pain in legs| Severe pain| 76948002| Severe pain|Severe pain:::Pain in lower limb:::Myalgia/myos...|
 42399005| kidney malfunction| Renal failure| 42399005| Renal failure|Renal failure:::Renal injury:::Kidney disease::...|
 288227007| muscle pain in my biceps| Myalgia/myositis - upper arm| 281245003| Musculoskeletal chest pain|Musculoskeletal chest pain:::Severe pain:::Myal...|
 419723007| brain fog| Mentally dull| 419723007| Mentally dull|Mentally dull:::Memory impairment:::Encephalopa...|
 248490000| bloating| Bloating symptom| 248490000| Bloating symptom|Bloating symptom:::Abdominal bloating:::Flatule...|
 22253000| pain| Pain| 22253000| Pain|Pain:::Foot pain:::Backache:::Severe pain:::Sho...|
 162076009| gas|Excessive upper gastrointestinal gas| 162076009| Excessive upper gastrointestinal gas|Excessive upper gastrointestinal gas:::Flatulen...|
 247592009| Short term memory loss| Poor short-term memory| 247592009| Poor short-term memory|Poor short-term memory:::Amnesia:::Memory impai...|
 62507009| tingling in my feet| Pins and needles| 309087008| Paraesthesia of foot|Paraesthesia of foot:::Pins and needles:::Cramp...|
3877011000036101| LIPITOR| Lipitor| 3877011000036101| Lipitor|Lipitor:::Lipitor 20 mg tablet: film-coated:::L...|
 62507009| tingling on left side of face| Pins and needles| 309557009| Numbness of face|Numbness of face:::Pins and needles:::Numbness:...|
+----------------+--------------------------------+------------------------------------+--------------------+-------------------------------------+--------------------------------------------------+
only showing top 20 rows

In [0]:
aap_test_pred.selectExpr("conceptId","term","_term","recognized[0].result","recognized[0].metadata.resolved_text","recognized[0].metadata.all_k_resolutions").show(truncate=50)

+----------------+--------------------------------+------------------------------------+--------------------+-------------------------------------+--------------------------------------------------+
 conceptId| term| _term|recognized[0].result|recognized[0].metadata[resolved_text]| recognized[0].metadata[all_k_resolutions]|
+----------------+--------------------------------+------------------------------------+--------------------+-------------------------------------+--------------------------------------------------+
 108367008| dislocating| Dislocation of joint| 304297005| Decreased range of shoulder movement|Decreased range of shoulder movement:::Increase...|
3384011000036100| Arthrotec| Arthrotec| 3384011000036100| Arthrotec|Arthrotec:::Celebrex 200 mg capsule: hard:::Cel...|
 166717003| elevated creatine| Serum creatinine raised| 166717003| Serum creatinine raised|Serum creatinine raised:::Increased creatine ki...|
3877011000036101| Lipitor| Lipitor| 3877011000036101| Lipitor|Lipitor:::Lipitor 20 mg tablet: film-coated:::L...|
 402234004| Eczema on feet| Foot eczema| 238539001| Hand eczema|Hand eczema:::Foot eczema:::Cold feet:::Blister...|
 404640003| dizzy| Dizziness| 404640003| Dizziness|Dizziness:::Lightheadedness:::Nausea:::Syncope:...|
 271681002| pain in the pit of my stomach| Stomach ache| 271681002| Stomach ache|Stomach ache:::Abdominal pain:::Myalgia:::Low b...|
 76948002|extreme muscle pain in shoulders| Severe pain| 76948002| Severe pain|Severe pain:::Myalgia/myositis - shoulder:::Mus...|
 36031001| Burning feet| Burning feet| 36031001| Burning feet|Burning feet:::Burning sensation:::Burning pain...|
 76948002| imence pain in legs| Severe pain| 76948002| Severe pain|Severe pain:::Pain in lower limb:::Myalgia/myos...|
 42399005| kidney malfunction| Renal failure| 42399005| Renal failure|Renal failure:::Renal injury:::Kidney disease::...|
 288227007| muscle pain in my biceps| Myalgia/myositis - upper arm| 281245003| Musculoskeletal chest pain|Musculoskeletal chest pain:::Severe pain:::Myal...|
 419723007| brain fog| Mentally dull| 419723007| Mentally dull|Mentally dull:::Memory impairment:::Encephalopa...|
 248490000| bloating| Bloating symptom| 248490000| Bloating symptom|Bloating symptom:::Abdominal bloating:::Flatule...|
 22253000| pain| Pain| 22253000| Pain|Pain:::Foot pain:::Backache:::Severe pain:::Sho...|
 162076009| gas|Excessive upper gastrointestinal gas| 162076009| Excessive upper gastrointestinal gas|Excessive upper gastrointestinal gas:::Flatulen...|
 247592009| Short term memory loss| Poor short-term memory| 247592009| Poor short-term memory|Poor short-term memory:::Amnesia:::Memory impai...|
 62507009| tingling in my feet| Pins and needles| 309087008| Paraesthesia of foot|Paraesthesia of foot:::Pins and needles:::Cramp...|
3877011000036101| LIPITOR| Lipitor| 3877011000036101| Lipitor|Lipitor:::Lipitor 20 mg tablet: film-coated:::L...|
 62507009| tingling on left side of face| Pins and needles| 309557009| Numbness of face|Numbness of face:::Pins and needles:::Numbness:...|
+----------------+--------------------------------+------------------------------------+--------------------+-------------------------------------+--------------------------------------------------+
only showing top 20 rows

## Train Using the entire dataset

In [0]:
all_data = aap_train_sdf.union(aap_test_sdf).union(aap_val_sdf)

snomed_training_model = snomed_training_pipeline.fit(all_data)

snomed_data = snomed_training_model.transform(all_data).cache()

%time model = snomed_extractor.fit(snomed_data)

CPU times: user 41.8 ms, sys: 19.3 ms, total: 61.1 ms
Wall time: 2.45 s

In [0]:
! cd /databricks/driver/ && mkdir models

In [0]:
model.write().overwrite().save("dbfs:/databricks/driver/models/chunkresolve_snomed_askapatient_hc_100d")

In [0]:
%sh cd /databricks/driver/models  && ls -la


total 8
drwxr-xr-x 2 root root 4096 Jul 10 18:51 .
drwxr-xr-x 1 root root 4096 Jul 10 18:51 ..

## Prediction on random texts

In [0]:
documentAssembler = DocumentAssembler()\
  .setInputCol("term")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")\
  .setCustomBounds([","])

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("raw_token")\

stopwords = StopWordsCleaner()\
  .setInputCols(["raw_token"])\
  .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_healthcare_100d", "en", "clinical/models")\
  .setInputCols(["document", "token"])\
  .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.pretrained("ner_healthcare", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

snomed_ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("greedy_chunk")\
  .setWhiteList(['PROBLEM','TEST'])

chunk_embeddings = ChunkEmbeddings()\
  .setInputCols('greedy_chunk', 'embeddings')\
  .setOutputCol('chunk_embeddings')

snomed_resolver = ChunkEntityResolverModel.load("dbfs:/databricks/driver/models/chunkresolve_snomed_askapatient_hc_100d")\
  .setInputCols("token","chunk_embeddings")\
  .setOutputCol("snomed_resolution")

pipeline_snomed = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    stopwords,
    word_embeddings,
    clinical_ner,
    snomed_ner_converter,
    chunk_embeddings,
    snomed_resolver
  ])

empty_data = spark.createDataFrame([['']]).toDF("term")

model_snomed = pipeline_snomed.fit(empty_data)


embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[ | ][OK!]
ner_healthcare download started this may take some time.
Approximate size to download 13.5 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
model_snomed_lp = LightPipeline(model_snomed)


In [0]:
result = model_snomed_lp.annotate('I have a biceps muscle pain and extreme muscle pain in shoulders')

list(zip(result['greedy_chunk'],result['snomed_resolution']))

Out[19]: [('biceps muscle pain', '288227007'), ('extreme muscle pain', '76948002')]

In [0]:
result = model_snomed_lp.annotate('I have a flu and a headache')

list(zip(result['greedy_chunk'],result['snomed_resolution']))

Out[20]: [('flu and a headache', '6142004')]

In [0]:
from pyspark.sql import functions as F

snomed_output = model_snomed.transform(spark.createDataFrame([['I have a biceps muscle pain and extreme muscle pain in shoulders']]).toDF("term"))

snomed_output.select(F.explode(F.arrays_zip("greedy_chunk.result","greedy_chunk.metadata","snomed_resolution.result","snomed_resolution.metadata")).alias("snomed_result")) \
             .select(F.expr("snomed_result['0']").alias("chunk"),
                     F.expr("snomed_result['1'].entity").alias("entity"),
                     F.expr("snomed_result['3'].all_k_resolutions").alias("target_text"),
                     F.expr("snomed_result['2']").alias("code"),
                     F.expr("snomed_result['3'].confidence").alias("confidence")).show(truncate = 100)

+-------------------+-------+----------------------------------------------------------------------------------------------------+---------+----------+
 chunk| entity| target_text| code|confidence|
+-------------------+-------+----------------------------------------------------------------------------------------------------+---------+----------+
 biceps muscle pain|PROBLEM|Myalgia/myositis - upper arm:::Myalgia:::Neck pain:::Myalgia/myositis - shoulder:::Backache:::Foo...|288227007| 0.0915|
extreme muscle pain|PROBLEM|Severe pain:::Muscle fatigue:::Muscle weakness:::Myopathy:::Abdominal pain:::Constant pain:::Myal...| 76948002| 0.1527|
+-------------------+-------+----------------------------------------------------------------------------------------------------+---------+----------+

# Sentence Entity Resolver (BioBert sentence embeddings) (after v2.7)

In [0]:
aap_train_sdf.show()

+----------------+--------------------+--------------------+
 conceptId| _term| term|
+----------------+--------------------+--------------------+
 108367008|Dislocation of joint|Dislocation of joint|
3384011000036100| Arthrotec| Arthrotec|
 166717003|Serum creatinine ...|Serum creatinine ...|
3877011000036101| Lipitor| Lipitor|
 402234004| Foot eczema| Foot eczema|
 404640003| Dizziness| Dizziness|
 271681002| Stomach ache| Stomach ache|
 76948002| Severe pain| Severe pain|
 36031001| Burning feet| Burning feet|
 76948002| Severe pain| Severe pain|
 42399005| Renal failure| Renal failure|
 288227007|Myalgia/myositis ...|Myalgia/myositis ...|
 419723007| Mentally dull| Mentally dull|
 248490000| Bloating symptom| Bloating symptom|
 22253000| Pain| Pain|
 162076009|Excessive upper g...|Excessive upper g...|
 247592009|Poor short-term m...|Poor short-term m...|
 62507009| Pins and needles| Pins and needles|
3877011000036101| Lipitor| Lipitor|
 62507009| Pins and needles| Pins and needles|
+----------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
aap_train_sdf.printSchema()

root
-- conceptId: string (nullable = true)
-- _term: string (nullable = true)
-- term: string (nullable = true)

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("_term")\
    .setOutputCol("sentence")

bert_embeddings = BertSentenceEmbeddings.pretrained("sent_biobert_pubmed_base_cased")\
    .setInputCols(["sentence"])\
    .setOutputCol("bert_embeddings")

snomed_training_pipeline = Pipeline(
    stages = [
    documentAssembler,
    bert_embeddings])


snomed_training_model = snomed_training_pipeline.fit(aap_train_sdf)
snomed_data = snomed_training_model.transform(aap_train_sdf)


sent_biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
bertExtractor = SentenceEntityResolverApproach()\
    .setNeighbours(25)\
    .setThreshold(1000)\
    .setInputCols("bert_embeddings")\
    .setNormalizedCol("_term")\
    .setLabelCol("conceptId")\
    .setOutputCol('snomed_code')\
    .setDistanceFunction("EUCLIDIAN")\
    .setCaseSensitive(False)

%time snomed_model = bertExtractor.fit(snomed_data)

CPU times: user 1.92 s, sys: 819 ms, total: 2.74 s
Wall time: 10min 6s

In [0]:
# save if you will need that later
model.write().overwrite().save("dbfs:/databricks/driver/models/biobertresolve_snomed_askapatient")

In [0]:
prediction_Model = PipelineModel(stages=[snomed_training_model, snomed_model])

aap_train_pred= prediction_Model.transform(aap_train_sdf).cache()
aap_test_pred= prediction_Model.transform(aap_test_sdf).cache()
aap_val_pred= prediction_Model.transform(aap_val_sdf).cache()

In [0]:
aap_test_pred.selectExpr("conceptId","term","_term","snomed_code[0].result","snomed_code[0].metadata.resolved_text","snomed_code[0].metadata.all_k_resolutions").show(truncate=50)

+----------------+--------------------------------+------------------------------------+---------------------+--------------------------------------+--------------------------------------------------+
 conceptId| term| _term|snomed_code[0].result|snomed_code[0].metadata[resolved_text]| snomed_code[0].metadata[all_k_resolutions]|
+----------------+--------------------------------+------------------------------------+---------------------+--------------------------------------+--------------------------------------------------+
 108367008| dislocating| Dislocation of joint| 108367008| Dislocation of joint|Dislocation of joint:::Disorder of joint of sho...|
3384011000036100| Arthrotec| Arthrotec| 3384011000036100| Arthrotec| Arthrotec|
 166717003| elevated creatine| Serum creatinine raised| 166717003| Serum creatinine raised|Serum creatinine raised:::Serum cholesterol raised|
3877011000036101| Lipitor| Lipitor| 3877011000036101| Lipitor| Lipitor|
 402234004| Eczema on feet| Foot eczema| 402234004| Foot eczema|Foot eczema:::Hand eczema:::Eczema:::Blistering...|
 404640003| dizzy| Dizziness| 404640003| Dizziness| Dizziness|
 271681002| pain in the pit of my stomach| Stomach ache| 271681002| Stomach ache| Stomach ache|
 76948002|extreme muscle pain in shoulders| Severe pain| 76948002| Severe pain| Severe pain|
 36031001| Burning feet| Burning feet| 36031001| Burning feet|Burning feet:::Cold feet:::Cold hands:::Burning...|
 76948002| imence pain in legs| Severe pain| 76948002| Severe pain| Severe pain|
 42399005| kidney malfunction| Renal failure| 42399005| Renal failure|Renal failure:::Renal injury:::Renal impairment...|
 288227007| muscle pain in my biceps| Myalgia/myositis - upper arm| 288227007| Myalgia/myositis - upper arm|Myalgia/myositis - upper arm:::Myalgia/myositis...|
 419723007| brain fog| Mentally dull| 419723007| Mentally dull| Mentally dull|
 248490000| bloating| Bloating symptom| 248490000| Bloating symptom| Bloating symptom|
 22253000| pain| Pain| 22253000| Pain| Pain|
 162076009| gas|Excessive upper gastrointestinal gas| 162076009| Excessive upper gastrointestinal gas| Excessive upper gastrointestinal gas|
 247592009| Short term memory loss| Poor short-term memory| 247592009| Poor short-term memory| Poor short-term memory|
 62507009| tingling in my feet| Pins and needles| 62507009| Pins and needles| Pins and needles|
3877011000036101| LIPITOR| Lipitor| 3877011000036101| Lipitor| Lipitor|
 62507009| tingling on left side of face| Pins and needles| 62507009| Pins and needles| Pins and needles|
+----------------+--------------------------------+------------------------------------+---------------------+--------------------------------------+--------------------------------------------------+
only showing top 20 rows

In [0]:
aap_val_pred.selectExpr("conceptId","term","_term","snomed_code[0].result","snomed_code[0].metadata.resolved_text","snomed_code[0].metadata.all_k_resolutions").show(truncate=50)

+----------------+------------------------------+---------------------+---------------------+--------------------------------------+--------------------------------------------------+
 conceptId| term| _term|snomed_code[0].result|snomed_code[0].metadata[resolved_text]| snomed_code[0].metadata[all_k_resolutions]|
+----------------+------------------------------+---------------------+---------------------+--------------------------------------+--------------------------------------------------+
 267032009|persisten feeling of tiredness| Tired all the time| 267032009| Tired all the time| Tired all the time:::Tightness in throat|
 22298006| HEART ATTACK|Myocardial infarction| 22298006| Myocardial infarction| Myocardial infarction|
3877011000036101| LIPITOR| Lipitor| 3877011000036101| Lipitor| Lipitor|
 415690000| sweated| Sweating| 415690000| Sweating| Sweating:::Sweating problem:::Swelling|
 248491001| swelling at knee| Swollen knee| 248491001| Swollen knee|Swollen knee:::Swollen ankle:::Swollen feet:::S...|
 248490000| bloating| Bloating symptom| 248490000| Bloating symptom| Bloating symptom|
 162059005| stomach upset| Upset stomach| 162059005| Upset stomach| Upset stomach|
3877011000036101| Lipitor| Lipitor| 3877011000036101| Lipitor| Lipitor|
 62507009| tingling| Pins and needles| 62507009| Pins and needles| Pins and needles|
 238131007| over weight| Overweight| 238131007| Overweight| Overweight:::Weight gain|
 73595000| stress| Stress| 73595000| Stress| Stress:::Stress overload:::Swelling|
3877011000036101| lipitor| Lipitor| 3877011000036101| Lipitor| Lipitor|
3877011000036101| Lipitor| Lipitor| 3877011000036101| Lipitor| Lipitor|
 22253000| hurt| Pain| 22253000| Pain| Pain|
 73211009| diabetes| Diabetes mellitus| 73211009| Diabetes mellitus| Diabetes mellitus|
 10601006| sore legs| Pain in lower limb| 10601006| Pain in lower limb| Pain in lower limb|
 75694006| acute pancreatic attack| Pancreatitis| 75694006| Pancreatitis|Pancreatitis:::Peritonitis:::Acute gastritis:::...|
 51197009| SEVERE abdominal cramping| Stomach cramps| 51197009| Stomach cramps| Stomach cramps|
 3723001| arthrites| Arthritis| 3723001| Arthritis| Arthritis|
 271767006| skin began to come off| Peeling of skin| 271767006| Peeling of skin|Peeling of skin:::Discolouration of skin:::Abra...|
+----------------+------------------------------+---------------------+---------------------+--------------------------------------+--------------------------------------------------+
only showing top 20 rows

End of Notebook #